## Using grid search to find the optimal hyperparameters for a Feedforward NN
<ul>
    <li>Keras is used to build the model of NN</li>
    <li>KerasClassifier in scikit-learn is used to build the model</li>
    <li>GridSearchCV from scikit-learn is used for searching the best set of hyperparameters</li>
</ul>

### Importing the libraries and function required

In [35]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score

### Loading the dataset
<ul>
    <li>Dataset is obtained from <a href="https://www.kaggle.com/uciml/pima-indians-diabetes-database">kaggle</a>     </li>
</ul>

In [16]:
names = ['n_pregnant', 'glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI', 'pedigree_function', 'age', 'class']
data = pd.read_csv("diabetes.csv", names = names, na_filter=True, header=0)

## EDA and pre-processing

In [18]:
data.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [19]:
# Dropping data with value = 0 for some columns
cols = ['glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI']
for col in cols:
    data = data[data[col] != 0]

In [20]:
data.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


### Converting dataframe into numpy array and splitting the data into input and output arrays

In [22]:
data_arr = data.values
X = data_arr[:, 0:8]
y = data_arr[:, 8].astype(int)

In [27]:
## Scaling the input data using standard scaler
X_scaled = StandardScaler().fit_transform(X)

### Grid search on batch_size and number of epochs

In [29]:
seed = 6
np.random.seed(seed)
def create_model():
    model = Sequential()
    model.add(Dense(8, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    opt = Adam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
model = KerasClassifier(build_fn=create_model, verbose = 1)
batch_size = [8, 16, 32]
epochs = [50, 100, 500]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=KFold(random_state=seed), verbose=1)
grid_res = grid.fit(X_scaled, y)

# Print the summary of resulting fit trials
print("Best score of {0} on params {1}".format(grid_res.best_score_, grid_res.best_params_))
mean_scores = grid_res.cv_results_["mean_test_score"]
std_scores = grid_res.cv_results_["std_test_score"]
params = grid_res.cv_results_["params"]
for param, mean_score, std_score in zip(params, mean_scores, std_scores):
    print("mean: {0} and std_dev: {1} with {2}".format(mean_score, std_score, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/50
261/261 [==============================] - 1s 5ms/step - loss: 0.6550 - acc: 0.7011
Epoch 2/50
261/261 [==============================] - 0s 122us/step - loss: 0.4886 - acc: 0.7778
Epoch 3/50
261/261 [==============================] - 0s 108us/step - loss: 0.4182 - acc: 0.7854
Epoch 4/50
261/261 [==============================] - 0s 140us/step - loss: 0.3934 - acc: 0.8161
Epoch 5/50
261/261 [==============================] - 0s 125us/step - loss: 0.3858 - acc: 0.8199
Epoch 6/50
261/261 [==============================] - 0s 124us/step - loss: 0.3758 - acc: 0.8391
Epoch 7/50
261/261 [==============================] - ETA: 0s - loss: 0.3316 - acc: 0.875 - 0s 116us/step - loss: 0.3764 - acc: 0.8238
Epoch 8/50
261/261 [==============================] - 0s 126us/step - loss: 0.3670 - acc: 0.8276
Epoch 9/50
261/261 [==============================] - 0s 127us/step - loss: 0.3664 - acc: 0.8391
Epoch 10/50
261/261 [==============================] - 0s 124us/step - loss: 0.3586 - acc: 

261/261 [==============================] - 0s 130us/step - loss: 0.3615 - acc: 0.8429
Epoch 33/50
261/261 [==============================] - 0s 124us/step - loss: 0.3674 - acc: 0.8352
Epoch 34/50
261/261 [==============================] - 0s 108us/step - loss: 0.3584 - acc: 0.8429
Epoch 35/50
261/261 [==============================] - 0s 109us/step - loss: 0.3637 - acc: 0.8467
Epoch 36/50
261/261 [==============================] - 0s 108us/step - loss: 0.3571 - acc: 0.8467
Epoch 37/50
261/261 [==============================] - 0s 111us/step - loss: 0.3528 - acc: 0.8506
Epoch 38/50
261/261 [==============================] - 0s 114us/step - loss: 0.3484 - acc: 0.8467
Epoch 39/50
261/261 [==============================] - 0s 120us/step - loss: 0.3572 - acc: 0.8544
Epoch 40/50
261/261 [==============================] - 0s 117us/step - loss: 0.3537 - acc: 0.8506
Epoch 41/50
261/261 [==============================] - 0s 111us/step - loss: 0.3444 - acc: 0.8467
Epoch 42/50
261/261 [===========

261/261 [==============================] - 0s 133us/step - loss: 0.3357 - acc: 0.8429
Epoch 14/100
261/261 [==============================] - 0s 126us/step - loss: 0.3296 - acc: 0.8506
Epoch 15/100
261/261 [==============================] - 0s 136us/step - loss: 0.3302 - acc: 0.8582
Epoch 16/100
261/261 [==============================] - 0s 128us/step - loss: 0.3272 - acc: 0.8506
Epoch 17/100
261/261 [==============================] - 0s 134us/step - loss: 0.3276 - acc: 0.8544
Epoch 18/100
261/261 [==============================] - 0s 139us/step - loss: 0.3191 - acc: 0.8544
Epoch 19/100
261/261 [==============================] - 0s 160us/step - loss: 0.3135 - acc: 0.8774
Epoch 20/100
261/261 [==============================] - 0s 141us/step - loss: 0.3157 - acc: 0.8582
Epoch 21/100
261/261 [==============================] - 0s 133us/step - loss: 0.3044 - acc: 0.8697
Epoch 22/100
261/261 [==============================] - 0s 139us/step - loss: 0.3116 - acc: 0.8659
Epoch 23/100
261/261 [=

...

262/262 [==============================] - 0s 57us/step - loss: 0.3209 - acc: 0.8664
Epoch 304/500
262/262 [==============================] - 0s 56us/step - loss: 0.3182 - acc: 0.8779
Epoch 305/500
262/262 [==============================] - 0s 62us/step - loss: 0.3070 - acc: 0.8893
Epoch 306/500
262/262 [==============================] - 0s 50us/step - loss: 0.3048 - acc: 0.8779
Epoch 307/500
262/262 [==============================] - 0s 48us/step - loss: 0.3067 - acc: 0.8855
Epoch 308/500
262/262 [==============================] - 0s 54us/step - loss: 0.3112 - acc: 0.8779
Epoch 309/500
262/262 [==============================] - 0s 56us/step - loss: 0.3080 - acc: 0.8740
Epoch 310/500
262/262 [==============================] - 0s 45us/step - loss: 0.3040 - acc: 0.8817
Epoch 311/500
262/262 [==============================] - 0s 50us/step - loss: 0.3041 - acc: 0.8817
Epoch 312/500
262/262 [==============================] - 0s 52us/step - loss: 0.3053 - acc: 0.8779
Epoch 313/500
262/262 [=

262/262 [==============================] - 0s 51us/step - loss: 0.2990 - acc: 0.8855
Epoch 386/500
262/262 [==============================] - 0s 58us/step - loss: 0.2968 - acc: 0.8855
Epoch 387/500
262/262 [==============================] - 0s 54us/step - loss: 0.2990 - acc: 0.8855
Epoch 388/500
262/262 [==============================] - 0s 58us/step - loss: 0.2950 - acc: 0.8855
Epoch 389/500
262/262 [==============================] - 0s 48us/step - loss: 0.2950 - acc: 0.8817
Epoch 390/500
262/262 [==============================] - 0s 50us/step - loss: 0.2984 - acc: 0.8817
Epoch 391/500
262/262 [==============================] - 0s 51us/step - loss: 0.2993 - acc: 0.8817
Epoch 392/500
262/262 [==============================] - 0s 60us/step - loss: 0.2971 - acc: 0.8817
Epoch 393/500
262/262 [==============================] - 0s 53us/step - loss: 0.2998 - acc: 0.8855
Epoch 394/500
262/262 [==============================] - 0s 46us/step - loss: 0.3031 - acc: 0.8779
Epoch 395/500
262/262 [=

262/262 [==============================] - 0s 52us/step - loss: 0.3111 - acc: 0.8817
Epoch 468/500
262/262 [==============================] - 0s 48us/step - loss: 0.3028 - acc: 0.8855
Epoch 469/500
262/262 [==============================] - 0s 53us/step - loss: 0.3009 - acc: 0.8779
Epoch 470/500
262/262 [==============================] - 0s 50us/step - loss: 0.3008 - acc: 0.8817
Epoch 471/500
262/262 [==============================] - 0s 58us/step - loss: 0.2957 - acc: 0.8779
Epoch 472/500
262/262 [==============================] - 0s 52us/step - loss: 0.2930 - acc: 0.8779
Epoch 473/500
262/262 [==============================] - 0s 58us/step - loss: 0.2940 - acc: 0.8779
Epoch 474/500
262/262 [==============================] - 0s 58us/step - loss: 0.2939 - acc: 0.8817
Epoch 475/500
262/262 [==============================] - 0s 48us/step - loss: 0.2913 - acc: 0.8779
Epoch 476/500
262/262 [==============================] - 0s 56us/step - loss: 0.2924 - acc: 0.8740
Epoch 477/500
262/262 [=

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  3.0min finished


Epoch 1/50
392/392 [==============================] - 1s 3ms/step - loss: 0.6735 - acc: 0.6556
Epoch 2/50
392/392 [==============================] - 0s 47us/step - loss: 0.5877 - acc: 0.6709
Epoch 3/50
392/392 [==============================] - 0s 45us/step - loss: 0.4969 - acc: 0.7449
Epoch 4/50
392/392 [==============================] - 0s 46us/step - loss: 0.4585 - acc: 0.7755
Epoch 5/50
392/392 [==============================] - 0s 46us/step - loss: 0.4447 - acc: 0.7704
Epoch 6/50
392/392 [==============================] - 0s 47us/step - loss: 0.4422 - acc: 0.7755
Epoch 7/50
392/392 [==============================] - 0s 45us/step - loss: 0.4372 - acc: 0.7832
Epoch 8/50
392/392 [==============================] - 0s 44us/step - loss: 0.4327 - acc: 0.7781
Epoch 9/50
392/392 [==============================] - 0s 51us/step - loss: 0.4367 - acc: 0.7730
Epoch 10/50
392/392 [==============================] - 0s 46us/step - loss: 0.4341 - acc: 0.7806
Epoch 11/50
392/392 [===================

### Grid search on learning_rate and dropout_rate
Values for batch_size and epochs are taken from the best fit in previous step

In [31]:
def create_model(learn_rate, dropout_rate):
    model = Sequential()
    model.add(Dense(8, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    opt = Adam(lr=learn_rate)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
model = KerasClassifier(build_fn=create_model, verbose = 1, epochs=50, batch_size=32)
learn_rate = [0.01, 0.03, 0.1, 0.3]
dropout_rate = [0., 0.1, 0.2, 0.3]
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=KFold(random_state=seed), verbose=1)
grid_res = grid.fit(X_scaled, y)

# Print the summary of resulting fit trials
print("Best score of {0} on params {1}".format(grid_res.best_score_, grid_res.best_params_))
mean_scores = grid_res.cv_results_["mean_test_score"]
std_scores = grid_res.cv_results_["std_test_score"]
params = grid_res.cv_results_["params"]
for param, mean_score, std_score in zip(params, mean_scores, std_scores):
    print("mean: {0} and std_dev: {1} with {2}".format(mean_score, std_score, param))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/50
261/261 [==============================] - 1s 6ms/step - loss: 0.6860 - acc: 0.6667
Epoch 2/50
261/261 [==============================] - 0s 62us/step - loss: 0.6547 - acc: 0.6973
Epoch 3/50
261/261 [==============================] - 0s 66us/step - loss: 0.5941 - acc: 0.6973
Epoch 4/50
261/261 [==============================] - 0s 64us/step - loss: 0.5180 - acc: 0.6973
Epoch 5/50
261/261 [==============================] - 0s 56us/step - loss: 0.4706 - acc: 0.6973
Epoch 6/50
261/261 [==============================] - 0s 66us/step - loss: 0.4539 - acc: 0.6973
Epoch 7/50
261/261 [==============================] - 0s 66us/step - loss: 0.4409 - acc: 0.6973
Epoch 8/50
261/261 [==============================] - 0s 60us/step - loss: 0.4298 - acc: 0.6973
Epoch 9/50
261/261 [==============================] - 0s 70us/step - loss: 0.4199 - acc: 0.6973
Epoch 10/50
261/261 [==============================] - 0s 67us/step - loss: 0.4143 - acc: 0.6973
Epoch 11/50
261/261 [===================

261/261 [==============================] - 0s 64us/step - loss: 0.3905 - acc: 0.8467
Epoch 35/50
261/261 [==============================] - 0s 67us/step - loss: 0.3890 - acc: 0.8467
Epoch 36/50
261/261 [==============================] - 0s 64us/step - loss: 0.3865 - acc: 0.8467
Epoch 37/50
261/261 [==============================] - 0s 68us/step - loss: 0.3828 - acc: 0.8352
Epoch 38/50
261/261 [==============================] - 0s 62us/step - loss: 0.3775 - acc: 0.8429
Epoch 39/50
261/261 [==============================] - 0s 70us/step - loss: 0.3758 - acc: 0.8582
Epoch 40/50
261/261 [==============================] - 0s 95us/step - loss: 0.3754 - acc: 0.8467
Epoch 41/50
261/261 [==============================] - 0s 63us/step - loss: 0.3699 - acc: 0.8544
Epoch 42/50
261/261 [==============================] - 0s 64us/step - loss: 0.3668 - acc: 0.8659
Epoch 43/50
261/261 [==============================] - 0s 66us/step - loss: 0.3683 - acc: 0.8544
Epoch 44/50
261/261 [=====================

261/261 [==============================] - 0s 76us/step - loss: 0.3344 - acc: 0.8544
Epoch 17/50
261/261 [==============================] - 0s 72us/step - loss: 0.3314 - acc: 0.8621
Epoch 18/50
261/261 [==============================] - 0s 78us/step - loss: 0.3073 - acc: 0.8851
Epoch 19/50
261/261 [==============================] - 0s 86us/step - loss: 0.2944 - acc: 0.8812
Epoch 20/50
261/261 [==============================] - 0s 106us/step - loss: 0.2911 - acc: 0.8889
Epoch 21/50
261/261 [==============================] - 0s 91us/step - loss: 0.3462 - acc: 0.8544
Epoch 22/50
261/261 [==============================] - 0s 89us/step - loss: 0.3369 - acc: 0.8659
Epoch 23/50
261/261 [==============================] - 0s 57us/step - loss: 0.3234 - acc: 0.8774
Epoch 24/50
261/261 [==============================] - 0s 59us/step - loss: 0.2938 - acc: 0.8774
Epoch 25/50
261/261 [==============================] - 0s 53us/step - loss: 0.2860 - acc: 0.8812
Epoch 26/50
261/261 [====================

...

262/262 [==============================] - 0s 81us/step - loss: 0.5302 - acc: 0.7214
Epoch 26/50
262/262 [==============================] - 0s 86us/step - loss: 0.5104 - acc: 0.7405
Epoch 27/50
262/262 [==============================] - 0s 87us/step - loss: 0.5447 - acc: 0.7252
Epoch 28/50
262/262 [==============================] - 0s 88us/step - loss: 0.5345 - acc: 0.7099
Epoch 29/50
262/262 [==============================] - 0s 83us/step - loss: 0.5469 - acc: 0.7176
Epoch 30/50
262/262 [==============================] - 0s 91us/step - loss: 0.5130 - acc: 0.7214
Epoch 31/50
262/262 [==============================] - 0s 88us/step - loss: 0.5272 - acc: 0.7366
Epoch 32/50
262/262 [==============================] - 0s 87us/step - loss: 0.5501 - acc: 0.7099
Epoch 33/50
262/262 [==============================] - 0s 99us/step - loss: 0.5514 - acc: 0.6794
Epoch 34/50
262/262 [==============================] - 0s 81us/step - loss: 0.5131 - acc: 0.7290
Epoch 35/50
262/262 [=====================

261/261 [==============================] - 0s 80us/step - loss: 0.6529 - acc: 0.6552
Epoch 8/50
261/261 [==============================] - 0s 91us/step - loss: 0.6459 - acc: 0.6552
Epoch 9/50
261/261 [==============================] - 0s 90us/step - loss: 0.6472 - acc: 0.6552
Epoch 10/50
261/261 [==============================] - 0s 84us/step - loss: 0.6464 - acc: 0.6552
Epoch 11/50
261/261 [==============================] - 0s 94us/step - loss: 0.6491 - acc: 0.6552
Epoch 12/50
261/261 [==============================] - 0s 88us/step - loss: 0.6473 - acc: 0.6552
Epoch 13/50
261/261 [==============================] - 0s 91us/step - loss: 0.6562 - acc: 0.6552
Epoch 14/50
261/261 [==============================] - 0s 93us/step - loss: 0.6600 - acc: 0.6552
Epoch 15/50
261/261 [==============================] - 0s 87us/step - loss: 0.6484 - acc: 0.6552
Epoch 16/50
261/261 [==============================] - 0s 90us/step - loss: 0.6481 - acc: 0.6552
Epoch 17/50
261/261 [=======================

262/262 [==============================] - 0s 88us/step - loss: 0.6596 - acc: 0.6527
Epoch 40/50
262/262 [==============================] - 0s 83us/step - loss: 0.6540 - acc: 0.6527
Epoch 41/50
262/262 [==============================] - 0s 96us/step - loss: 0.6552 - acc: 0.6527
Epoch 42/50
262/262 [==============================] - 0s 96us/step - loss: 0.6478 - acc: 0.6527
Epoch 43/50
262/262 [==============================] - 0s 82us/step - loss: 0.6551 - acc: 0.6527
Epoch 44/50
262/262 [==============================] - 0s 91us/step - loss: 0.6487 - acc: 0.6527
Epoch 45/50
262/262 [==============================] - 0s 100us/step - loss: 0.6525 - acc: 0.6527
Epoch 46/50
262/262 [==============================] - 0s 90us/step - loss: 0.6483 - acc: 0.6527
Epoch 47/50
262/262 [==============================] - 0s 90us/step - loss: 0.6480 - acc: 0.6527
Epoch 48/50
262/262 [==============================] - 0s 85us/step - loss: 0.6496 - acc: 0.6527
Epoch 49/50
262/262 [====================

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:  5.1min finished
/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/50
392/392 [==============================] - 6s 15ms/step - loss: 0.6648 - acc: 0.6939
Epoch 2/50
392/392 [==============================] - 0s 83us/step - loss: 0.5776 - acc: 0.7500
Epoch 3/50
392/392 [==============================] - 0s 82us/step - loss: 0.5072 - acc: 0.7577
Epoch 4/50
392/392 [==============================] - 0s 77us/step - loss: 0.5112 - acc: 0.7679
Epoch 5/50
392/392 [==============================] - 0s 82us/step - loss: 0.5011 - acc: 0.7730
Epoch 6/50
392/392 [==============================] - 0s 77us/step - loss: 0.4939 - acc: 0.7679
Epoch 7/50
392/392 [==============================] - 0s 85us/step - loss: 0.4692 - acc: 0.7934
Epoch 8/50
392/392 [==============================] - 0s 84us/step - loss: 0.4729 - acc: 0.7857
Epoch 9/50
392/392 [==============================] - 0s 77us/step - loss: 0.4577 - acc: 0.8087
Epoch 10/50
392/392 [==============================] - 0s 84us/step - loss: 0.4727 - acc: 0.7832
Epoch 11/50
392/392 [==================

## Grid search for optimal kernel_initializer and activation_function
The values for learn_rate and dropout_rate are chosen from the previous step

In [33]:
def create_model(activation, init):
    model = Sequential()
    model.add(Dense(8, input_dim=8, kernel_initializer=init, activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(4, kernel_initializer=init, activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    opt = Adam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
model = KerasClassifier(build_fn=create_model, verbose = 1, epochs=50, batch_size=32)
init = ['lecun_normal', 'glorot_normal', 'he_normal', 'lecun_uniform', 'glorot_uniform', 'he_uniform']
activation = ['sigmoid', 'tanh', 'relu', 'linear']
param_grid = dict(activation=activation, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=KFold(random_state=seed), verbose=1)
grid_res = grid.fit(X_scaled, y)

# Print the summary of resulting fit trials
print("Best score of {0} on params {1}".format(grid_res.best_score_, grid_res.best_params_))
mean_scores = grid_res.cv_results_["mean_test_score"]
std_scores = grid_res.cv_results_["std_test_score"]
params = grid_res.cv_results_["params"]
for param, mean_score, std_score in zip(params, mean_scores, std_scores):
    print("mean: {0} and std_dev: {1} with {2}".format(mean_score, std_score, param))

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/50
261/261 [==============================] - 6s 24ms/step - loss: 0.6329 - acc: 0.6782
Epoch 2/50
261/261 [==============================] - 0s 158us/step - loss: 0.6121 - acc: 0.6858
Epoch 3/50
261/261 [==============================] - 0s 108us/step - loss: 0.6051 - acc: 0.6973
Epoch 4/50
261/261 [==============================] - 0s 114us/step - loss: 0.6025 - acc: 0.6973
Epoch 5/50
261/261 [==============================] - 0s 113us/step - loss: 0.5762 - acc: 0.6973
Epoch 6/50
261/261 [==============================] - 0s 115us/step - loss: 0.5792 - acc: 0.6973
Epoch 7/50
261/261 [==============================] - 0s 114us/step - loss: 0.5668 - acc: 0.6973
Epoch 8/50
261/261 [==============================] - 0s 122us/step - loss: 0.5392 - acc: 0.6935
Epoch 9/50
261/261 [==============================] - 0s 116us/step - loss: 0.5391 - acc: 0.7088
Epoch 10/50
261/261 [==============================] - 0s 115us/step - loss: 0.5354 - acc: 0.7050
Epoch 11/50
261/261 [=========

261/261 [==============================] - 0s 117us/step - loss: 0.4526 - acc: 0.7739
Epoch 34/50
261/261 [==============================] - 0s 123us/step - loss: 0.4582 - acc: 0.7625
Epoch 35/50
261/261 [==============================] - 0s 94us/step - loss: 0.4670 - acc: 0.7625
Epoch 36/50
261/261 [==============================] - 0s 93us/step - loss: 0.4787 - acc: 0.7433
Epoch 37/50
261/261 [==============================] - 0s 92us/step - loss: 0.4617 - acc: 0.7739
Epoch 38/50
261/261 [==============================] - 0s 99us/step - loss: 0.4463 - acc: 0.7854
Epoch 39/50
261/261 [==============================] - 0s 93us/step - loss: 0.4864 - acc: 0.7548
Epoch 40/50
261/261 [==============================] - 0s 89us/step - loss: 0.4728 - acc: 0.7663
Epoch 41/50
261/261 [==============================] - 0s 107us/step - loss: 0.4418 - acc: 0.8008
Epoch 42/50
261/261 [==============================] - 0s 91us/step - loss: 0.4618 - acc: 0.7625
Epoch 43/50
261/261 [==================

261/261 [==============================] - 0s 78us/step - loss: 0.5314 - acc: 0.7433
Epoch 16/50
261/261 [==============================] - 0s 77us/step - loss: 0.4975 - acc: 0.7816
Epoch 17/50
261/261 [==============================] - 0s 75us/step - loss: 0.4975 - acc: 0.8008
Epoch 18/50
261/261 [==============================] - 0s 75us/step - loss: 0.5177 - acc: 0.7778
Epoch 19/50
261/261 [==============================] - 0s 76us/step - loss: 0.5010 - acc: 0.7586
Epoch 20/50
261/261 [==============================] - 0s 76us/step - loss: 0.4816 - acc: 0.7778
Epoch 21/50
261/261 [==============================] - 0s 75us/step - loss: 0.4976 - acc: 0.7701
Epoch 22/50
261/261 [==============================] - 0s 75us/step - loss: 0.4795 - acc: 0.7663
Epoch 23/50
261/261 [==============================] - 0s 78us/step - loss: 0.4608 - acc: 0.8008
Epoch 24/50
261/261 [==============================] - 0s 77us/step - loss: 0.4744 - acc: 0.7969
Epoch 25/50
261/261 [=====================

...

262/262 [==============================] - 0s 162us/step - loss: 0.4997 - acc: 0.7443
Epoch 26/50
262/262 [==============================] - 0s 159us/step - loss: 0.4969 - acc: 0.7557
Epoch 27/50
262/262 [==============================] - 0s 152us/step - loss: 0.5036 - acc: 0.7557
Epoch 28/50
262/262 [==============================] - 0s 152us/step - loss: 0.4800 - acc: 0.7710
Epoch 29/50
262/262 [==============================] - 0s 147us/step - loss: 0.4905 - acc: 0.7557
Epoch 30/50
262/262 [==============================] - 0s 147us/step - loss: 0.4976 - acc: 0.7634
Epoch 31/50
262/262 [==============================] - 0s 149us/step - loss: 0.4855 - acc: 0.7634
Epoch 32/50
262/262 [==============================] - 0s 149us/step - loss: 0.4936 - acc: 0.7710
Epoch 33/50
262/262 [==============================] - 0s 151us/step - loss: 0.4875 - acc: 0.7710
Epoch 34/50
262/262 [==============================] - 0s 152us/step - loss: 0.4908 - acc: 0.7405
Epoch 35/50
262/262 [===========

261/261 [==============================] - 0s 146us/step - loss: 0.6121 - acc: 0.7165
Epoch 7/50
261/261 [==============================] - 0s 143us/step - loss: 0.5367 - acc: 0.7548
Epoch 8/50
261/261 [==============================] - 0s 133us/step - loss: 0.5288 - acc: 0.7510
Epoch 9/50
261/261 [==============================] - 0s 129us/step - loss: 0.4667 - acc: 0.7778
Epoch 10/50
261/261 [==============================] - 0s 125us/step - loss: 0.5062 - acc: 0.7739
Epoch 11/50
261/261 [==============================] - 0s 134us/step - loss: 0.4577 - acc: 0.7854
Epoch 12/50
261/261 [==============================] - 0s 127us/step - loss: 0.4815 - acc: 0.7816
Epoch 13/50
261/261 [==============================] - 0s 132us/step - loss: 0.5058 - acc: 0.7586
Epoch 14/50
261/261 [==============================] - 0s 137us/step - loss: 0.4731 - acc: 0.7854
Epoch 15/50
261/261 [==============================] - 0s 123us/step - loss: 0.4830 - acc: 0.7701
Epoch 16/50
261/261 [==============

262/262 [==============================] - 0s 127us/step - loss: 0.4875 - acc: 0.7634
Epoch 39/50
262/262 [==============================] - 0s 133us/step - loss: 0.4861 - acc: 0.7634
Epoch 40/50
262/262 [==============================] - 0s 132us/step - loss: 0.5018 - acc: 0.7481
Epoch 41/50
262/262 [==============================] - 0s 131us/step - loss: 0.4597 - acc: 0.7748
Epoch 42/50
262/262 [==============================] - 0s 124us/step - loss: 0.5024 - acc: 0.7595
Epoch 43/50
262/262 [==============================] - 0s 135us/step - loss: 0.4719 - acc: 0.7748
Epoch 44/50
262/262 [==============================] - 0s 126us/step - loss: 0.4894 - acc: 0.7824
Epoch 45/50
262/262 [==============================] - 0s 130us/step - loss: 0.4788 - acc: 0.7519
Epoch 46/50
262/262 [==============================] - 0s 133us/step - loss: 0.5088 - acc: 0.7634
Epoch 47/50
262/262 [==============================] - 0s 131us/step - loss: 0.5158 - acc: 0.7672
Epoch 48/50
262/262 [===========

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 20.9min finished
/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/50
392/392 [==============================] - 14s 34ms/step - loss: 0.6268 - acc: 0.6531
Epoch 2/50
392/392 [==============================] - 0s 116us/step - loss: 0.5518 - acc: 0.7194
Epoch 3/50
392/392 [==============================] - 0s 116us/step - loss: 0.5329 - acc: 0.7372
Epoch 4/50
392/392 [==============================] - 0s 119us/step - loss: 0.5035 - acc: 0.7755
Epoch 5/50
392/392 [==============================] - 0s 118us/step - loss: 0.4930 - acc: 0.7704
Epoch 6/50
392/392 [==============================] - 0s 116us/step - loss: 0.4515 - acc: 0.7959
Epoch 7/50
392/392 [==============================] - 0s 119us/step - loss: 0.4816 - acc: 0.7755
Epoch 8/50
392/392 [==============================] - 0s 120us/step - loss: 0.4673 - acc: 0.7832
Epoch 9/50
392/392 [==============================] - 0s 119us/step - loss: 0.4610 - acc: 0.7908
Epoch 10/50
392/392 [==============================] - 0s 119us/step - loss: 0.4702 - acc: 0.7985
Epoch 11/50
392/392 [========

## Grid search for optimal number of neurons for hidden layers
kernel_initializer and activation are taken from the best performing in previous step

In [34]:
def create_model(neuron_1, neuron_2):
    model = Sequential()
    model.add(Dense(neuron_1, input_dim=8, kernel_initializer='glorot_uniform', activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(neuron_2, kernel_initializer='glorot_uniform', activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    opt = Adam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
model = KerasClassifier(build_fn=create_model, verbose = 1, epochs=50, batch_size=32)
neuron_1 = [4, 8, 16]
neuron_2 = [2, 4, 8]
param_grid = dict(neuron_1=neuron_1, neuron_2=neuron_2)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=KFold(random_state=seed), verbose=1)
grid_res = grid.fit(X_scaled, y)

# Print the summary of resulting fit trials
print("Best score of {0} on params {1}".format(grid_res.best_score_, grid_res.best_params_))
mean_scores = grid_res.cv_results_["mean_test_score"]
std_scores = grid_res.cv_results_["std_test_score"]
params = grid_res.cv_results_["params"]
for param, mean_score, std_score in zip(params, mean_scores, std_scores):
    print("mean: {0} and std_dev: {1} with {2}".format(mean_score, std_score, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/50
261/261 [==============================] - 14s 53ms/step - loss: 0.6373 - acc: 0.6360
Epoch 2/50
261/261 [==============================] - 0s 154us/step - loss: 0.6065 - acc: 0.6398
Epoch 3/50
261/261 [==============================] - 0s 154us/step - loss: 0.5641 - acc: 0.7318
Epoch 4/50
261/261 [==============================] - 0s 146us/step - loss: 0.5230 - acc: 0.7318
Epoch 5/50
261/261 [==============================] - 0s 156us/step - loss: 0.5311 - acc: 0.7510
Epoch 6/50
261/261 [==============================] - 0s 162us/step - loss: 0.4917 - acc: 0.7625
Epoch 7/50
261/261 [==============================] - 0s 146us/step - loss: 0.5005 - acc: 0.7548
Epoch 8/50
261/261 [==============================] - 0s 168us/step - loss: 0.4667 - acc: 0.8008
Epoch 9/50
261/261 [==============================] - 0s 153us/step - loss: 0.4735 - acc: 0.7663
Epoch 10/50
261/261 [==============================] - 0s 153us/step - loss: 0.4753 - acc: 0.7701
Epoch 11/50
261/261 [========

261/261 [==============================] - 0s 160us/step - loss: 0.4378 - acc: 0.8276
Epoch 34/50
261/261 [==============================] - 0s 166us/step - loss: 0.4755 - acc: 0.7969
Epoch 35/50
261/261 [==============================] - 0s 139us/step - loss: 0.4429 - acc: 0.8084
Epoch 36/50
261/261 [==============================] - 0s 144us/step - loss: 0.4305 - acc: 0.7969
Epoch 37/50
261/261 [==============================] - 0s 137us/step - loss: 0.4409 - acc: 0.7969
Epoch 38/50
261/261 [==============================] - 0s 137us/step - loss: 0.4522 - acc: 0.7854
Epoch 39/50
261/261 [==============================] - 0s 144us/step - loss: 0.4383 - acc: 0.8084
Epoch 40/50
261/261 [==============================] - 0s 145us/step - loss: 0.4239 - acc: 0.8467
Epoch 41/50
261/261 [==============================] - 0s 142us/step - loss: 0.4559 - acc: 0.8123
Epoch 42/50
261/261 [==============================] - 0s 174us/step - loss: 0.4551 - acc: 0.7854
Epoch 43/50
261/261 [===========

261/261 [==============================] - 0s 164us/step - loss: 0.4117 - acc: 0.8008
Epoch 15/50
261/261 [==============================] - 0s 149us/step - loss: 0.4248 - acc: 0.8008
Epoch 16/50
261/261 [==============================] - 0s 149us/step - loss: 0.4204 - acc: 0.8238
Epoch 17/50
261/261 [==============================] - 0s 142us/step - loss: 0.4132 - acc: 0.8199
Epoch 18/50
261/261 [==============================] - 0s 137us/step - loss: 0.4313 - acc: 0.8084
Epoch 19/50
261/261 [==============================] - 0s 142us/step - loss: 0.4242 - acc: 0.8123
Epoch 20/50
261/261 [==============================] - 0s 143us/step - loss: 0.4189 - acc: 0.8046
Epoch 21/50
261/261 [==============================] - 0s 135us/step - loss: 0.3899 - acc: 0.8084
Epoch 22/50
261/261 [==============================] - 0s 131us/step - loss: 0.4068 - acc: 0.8238
Epoch 23/50
261/261 [==============================] - 0s 142us/step - loss: 0.4161 - acc: 0.8008
Epoch 24/50
261/261 [===========

...

261/261 [==============================] - 0s 161us/step - loss: 0.4106 - acc: 0.8199
Epoch 42/50
261/261 [==============================] - 0s 162us/step - loss: 0.3787 - acc: 0.8352
Epoch 43/50
261/261 [==============================] - 0s 153us/step - loss: 0.4026 - acc: 0.8123
Epoch 44/50
261/261 [==============================] - 0s 148us/step - loss: 0.4020 - acc: 0.8123
Epoch 45/50
261/261 [==============================] - 0s 153us/step - loss: 0.4073 - acc: 0.8123
Epoch 46/50
261/261 [==============================] - 0s 149us/step - loss: 0.4108 - acc: 0.8084
Epoch 47/50
261/261 [==============================] - 0s 154us/step - loss: 0.3712 - acc: 0.8238
Epoch 48/50
261/261 [==============================] - 0s 160us/step - loss: 0.3746 - acc: 0.8429
Epoch 49/50
261/261 [==============================] - 0s 150us/step - loss: 0.3859 - acc: 0.8429
Epoch 50/50
261/261 [==============================] - 0s 130us/step
Epoch 1/50
262/262 [==============================] - 17s 64m

261/261 [==============================] - 0s 180us/step - loss: 0.3735 - acc: 0.8276
Epoch 23/50
261/261 [==============================] - 0s 166us/step - loss: 0.3648 - acc: 0.8467
Epoch 24/50
261/261 [==============================] - 0s 156us/step - loss: 0.3786 - acc: 0.8199
Epoch 25/50
261/261 [==============================] - 0s 143us/step - loss: 0.3792 - acc: 0.8238
Epoch 26/50
261/261 [==============================] - 0s 155us/step - loss: 0.3679 - acc: 0.8276
Epoch 27/50
261/261 [==============================] - 0s 152us/step - loss: 0.3851 - acc: 0.8238
Epoch 28/50
261/261 [==============================] - 0s 158us/step - loss: 0.3897 - acc: 0.8238
Epoch 29/50
261/261 [==============================] - 0s 150us/step - loss: 0.3743 - acc: 0.8238
Epoch 30/50
261/261 [==============================] - 0s 155us/step - loss: 0.3664 - acc: 0.8429
Epoch 31/50
261/261 [==============================] - 0s 165us/step - loss: 0.3890 - acc: 0.8199
Epoch 32/50
261/261 [===========

262/262 [==============================] - 0s 201us/step - loss: 0.5052 - acc: 0.7557
Epoch 4/50
262/262 [==============================] - 0s 181us/step - loss: 0.4953 - acc: 0.7595
Epoch 5/50
262/262 [==============================] - 0s 157us/step - loss: 0.5073 - acc: 0.7405
Epoch 6/50
262/262 [==============================] - 0s 159us/step - loss: 0.4929 - acc: 0.7481
Epoch 7/50
262/262 [==============================] - 0s 152us/step - loss: 0.4720 - acc: 0.7672
Epoch 8/50
262/262 [==============================] - 0s 162us/step - loss: 0.4715 - acc: 0.7786
Epoch 9/50
262/262 [==============================] - 0s 157us/step - loss: 0.4865 - acc: 0.7481
Epoch 10/50
262/262 [==============================] - 0s 146us/step - loss: 0.4718 - acc: 0.7748
Epoch 11/50
262/262 [==============================] - 0s 160us/step - loss: 0.4824 - acc: 0.7481
Epoch 12/50
262/262 [==============================] - 0s 150us/step - loss: 0.4936 - acc: 0.7481
Epoch 13/50
262/262 [=================

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 11.0min finished
/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/50
392/392 [==============================] - 17s 44ms/step - loss: 0.6281 - acc: 0.6607
Epoch 2/50
392/392 [==============================] - 0s 144us/step - loss: 0.5342 - acc: 0.7755
Epoch 3/50
392/392 [==============================] - 0s 143us/step - loss: 0.4945 - acc: 0.7908
Epoch 4/50
392/392 [==============================] - 0s 144us/step - loss: 0.4891 - acc: 0.7883
Epoch 5/50
392/392 [==============================] - 0s 142us/step - loss: 0.4711 - acc: 0.7781
Epoch 6/50
392/392 [==============================] - 0s 146us/step - loss: 0.4727 - acc: 0.7730
Epoch 7/50
392/392 [==============================] - 0s 141us/step - loss: 0.4686 - acc: 0.7959
Epoch 8/50
392/392 [==============================] - 0s 141us/step - loss: 0.4687 - acc: 0.7679
Epoch 9/50
392/392 [==============================] - 0s 144us/step - loss: 0.4628 - acc: 0.7883
Epoch 10/50
392/392 [==============================] - 0s 140us/step - loss: 0.4606 - acc: 0.7781
Epoch 11/50
392/392 [========

## Evaluating the final model

In [36]:
y_pred = grid.predict(X_scaled)
print(accuracy_score(y, y_pred))
print(classification_report(y, y_pred))

392/392 [==============================] - 8s 21ms/step
0.8367346938775511
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       262
           1       0.78      0.72      0.74       130

   micro avg       0.84      0.84      0.84       392
   macro avg       0.82      0.81      0.81       392
weighted avg       0.83      0.84      0.83       392

